In [1]:
# IMPORT LIBRARIES
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

In [2]:
# DATA PREPARATION 
df = pd.read_csv("initial_dataset_sample.csv")

# Convert date column
df['Hire_Date'] = pd.to_datetime(df['Hire_Date'], errors='coerce')

# Calculate turnover metrics
total_employees = len(df)
exited = df[df['Status'].str.lower().isin(['resigned', 'terminated', 'left'])].shape[0]
overall_turnover = (exited / total_employees) * 100

# Turnover by department
turnover_by_dept = (
    df.groupby('Department')['Status']
      .apply(lambda x: (x.str.lower().isin(['resigned', 'terminated', 'left']).sum() / len(x)) * 100)
      .reset_index(name='TurnoverRate')
)



In [3]:
# DASH APP LAYOUT
app = Dash(__name__)
app.title = "Employee Analytics Dashboard"

app.layout = html.Div([
    html.H2("Employee Analytics Dashboard", style={'textAlign': 'center'}),

    html.Div([
        html.P(f"Overall Turnover Rate: {overall_turnover:.1f}%", style={'fontSize': '18px'}),
    ], style={'textAlign': 'center'}),

    html.Div([
        
        dcc.Dropdown(
            id='dept-dropdown',
            options=[{'label': dept, 'value': dept} for dept in sorted(df['Department'].unique())],
            value=None,
            placeholder='Select Department (optional)'
        )
    ], style={'width': '40%', 'margin': 'auto'}),

    dcc.Graph(id='turnover-graph'),
    dcc.Graph(id='salary-graph'),
    dcc.Graph(id='experience-performance-graph')
])

In [4]:
# CALLBACKS
@app.callback(
    [Output('turnover-graph', 'figure'),
     Output('salary-graph', 'figure'),
     Output('experience-performance-graph', 'figure')],
    Input('dept-dropdown', 'value')
)
def update_graphs(selected_dept):
    # -------------------------------------------------------------------------------------------------------- TURNOVER CHART 
    if selected_dept:
        filtered = df[df['Department'] == selected_dept]
        turnover = (
            filtered.groupby('Job_Level')['Status']
                .apply(lambda x: (x.str.lower().isin(['resigned', 'terminated', 'left']).sum() / len(x)) * 100)
                .reset_index(name='TurnoverRate')
        )
        fig_turnover = px.bar(
            turnover, x='Job_Level', y='TurnoverRate',
            title=f'Turnover Rate by Job Level – {selected_dept}',
            text=turnover['TurnoverRate'].apply(lambda x: f"{x:.1f}%")
        )
    else:
        fig_turnover = px.bar(
            turnover_by_dept, x='Department', y='TurnoverRate',
            title='Turnover Rate by Department',
            text=turnover_by_dept['TurnoverRate'].apply(lambda x: f"{x:.1f}%")
        )

    fig_turnover.update_traces(textposition='outside')
    fig_turnover.update_layout(yaxis_title='Turnover Rate (%)', xaxis_title='')

    # ----------------------------------------------------------------------------------------- SALARY CHART
    if selected_dept:
        filtered_salary = df[df['Department'] == selected_dept]
        fig_salary = px.line(
            filtered_salary.sort_values('Hire_Date'),
            x='Hire_Date', y='Salary_INR',
            title=f'Salary Trend Over Time – {selected_dept}',
            markers=True
        )
    else:
        avg_salary = df.groupby('Department', as_index=False)['Salary_INR'].mean()
        fig_salary = px.bar(
            avg_salary, x='Department', y='Salary_INR',
            title='Average Salary by Department',
            text=avg_salary['Salary_INR'].apply(lambda x: f"${x:,.0f}")
        )
        fig_salary.update_traces(textposition='outside')

    # Format y-axis as currency
    fig_salary.update_layout(
        yaxis_title='Salary (USD)',
        xaxis_title='',
        yaxis_tickprefix="$",
        yaxis_tickformat=",",
    )

    # ----------------------------------------------------------------------------------------- EXPERIENCE VS PERFORMANCE CHART
    if selected_dept:
        filtered_exp = df[df['Department'] == selected_dept]

        # Group by Job Title
        grouped = (
            filtered_exp.groupby(['Job_Title', 'Job_Level'], as_index=False)
            .agg({
                'Experience_Years': 'mean',
                'Performance_Rating': 'mean',
                'Salary_INR': 'mean'
            })
        )

        fig_exp_perf = px.scatter(
            grouped,
            x='Experience_Years',
            y='Performance_Rating',
            color='Job_Level',
            size='Salary_INR',
            title=f'Experience vs Performance by Job Title – {selected_dept}',
            hover_data={'Job_Title': True,
                        'Experience_Years': ':.1f',
                        'Performance_Rating': ':.1f',
                        'Salary_INR': ':$,'
                       }
        )

    else:
        # Group across departments
        grouped = (
            df.groupby(['Department', 'Job_Title'], as_index=False)
            .agg({
                'Experience_Years': 'mean',
                'Performance_Rating': 'mean',
                'Salary_INR': 'mean'
            })
        )

        fig_exp_perf = px.scatter(
            grouped,
            x='Experience_Years',
            y='Performance_Rating',
            color='Department',
            size='Salary_INR',
            title='Experience vs Performance by Job Title (All Departments)',
            hover_data={'Job_Title': True,
                        'Department': True,
                        'Experience_Years': ':.1f',
                        'Performance_Rating': ':.1f',
                        'Salary_INR': ':$,'
                       }
        )

    fig_exp_perf.update_layout(
        xaxis_title='Average Years of Experience',
        yaxis_title='Average Performance Rating (1–5)',
        plot_bgcolor='white'
    )


    # ----------------------------------------------------------------------------------------- END    
    
    return fig_turnover, fig_salary, fig_exp_perf

In [5]:
# RUN SERVER
if __name__ == "__main__":
    # app.run_server(debug=True, mode="external")
    app.run(debug=True, jupyter_mode='external', port=8050)

Dash app running on http://127.0.0.1:8050/
